In [1]:
import pandas as pd
import numpy as np

In [2]:
df =  pd.read_csv('../Bases/Criados/4_base_preenchida.csv')

In [3]:
df = df.drop(columns=['B001', 'E006011', 'E008', 'J01101', 'M00203', 'P02001', 'P02002','P02501', 'P02602', 'P06701', 'T001', 'H001', 'H002', 'H003', 'H004','H010', 'N00101',
                      'Temporariamente_afastado', 'Mobilidade_auxiliar', 'Exposicao_quimica', 'Exposicao_ruido', 'Exposicao_lixo', 'Exposicao_minerio', 'Doses_semanais',
                      'Plano_publico', 'Cobertura_plano', 'P068', 'Q06307', 'Q06308', 'Q06309', 'Q06310', 'Q06311'])

In [181]:
#Mantém apenas as linhas onde C009 está entre 1 e 5;
#Reseta o índice, para que a base fique com numeração sequencial limpa.
df = [df['C009'].isin([1, 2, 3, 4, 5])].reset_index(drop=True)

AttributeError: 'list' object has no attribute 'reset_index'

In [4]:
#Renda

df['F001021'] = df.apply(lambda x: x['F001021'] if x['F001011'] == 1 else 0, axis=1)
df['F007021'] = df.apply(lambda x: x['F007021'] if x['F007011'] == 1 else 0, axis=1)
df['F008021'] = df.apply(lambda x: x['F008021'] if x['F008011'] == 1 else 0, axis=1)
df['VDF00102'] = df.apply(lambda x: x['VDF00102'] if x['VDF001'] == 1 else 0, axis=1)

# Somando tudo com a renda
df['Renda'] = df['Renda'] + df['F001021'] + df['F007021'] + df['F008021'] + df['VDF00102']

df = df.drop('F001021', axis=1)
df = df.drop('F007021', axis=1)
df = df.drop('F008021', axis=1)
df = df.drop('VDF00102', axis=1)
df = df.drop('F001011', axis=1)
df = df.drop('F007011', axis=1)
df = df.drop('F008011', axis=1)
df = df.drop('VDF001', axis=1)

#### lalalalala

1. J002- Nas duas últimas semanas, ___ deixou de realizar quaisquer de suas atividades habituais (trabalhar, ir à escola, brincar, afazeres domésticos etc.) por motivo da própria saúde
2. J00402- Qual foi o principal motivo de saúde que impediu ___ de realizar suas atividades habituais nas duas últimas semanas
3. J00404- Este motivo de saúde estava relacionado ao trabalho

se J002 = 2, então 0.

se J002 = 1 e J00402 = 8 e J00404 = 1, então 1.

se J002 = 1 e J00402 = 8 e J00404 = 2, então 2.


In [5]:
condicoes = [
    df['J002'] == 2,
    (df['J002'] == 1) & (df['J00402'] == 8) & (df['J00404'] == 1),
    (df['J002'] == 1) & (df['J00402'] == 8) & (df['J00404'] == 2)
]

# Resultados correspondentes às condições
valores = [0, 1, 2]

# Crie o novo atributo
df['AusenciaTrabalhoPorDC'] = np.select(condicoes, valores, default=np.nan)

In [6]:
print(df['AusenciaTrabalhoPorDC'].isna().sum())


2639


In [7]:
df['AusenciaTrabalhoPorDC'] = df['AusenciaTrabalhoPorDC'].fillna(0)

In [8]:
df = df.drop(columns=['J001', 'J002', 'J00402', 'J00404'])

#### lalalla

1. J007 -  Algum médico já deu o diagnóstico de alguma doença  crônica, física ou mental, ou doença de longa duração (de mais de 6 meses de duração) a  ___ ?
2. J00801 - Alguma dessas doenças limita de alguma forma suas atividades habituais (trabalhar, ir à escola, brincar, afazeres domésticos, etc.)

se J007 = 2, então 0.

se J007 = 1 e J00801 = 1, então 1.

se J007 = 1 e J00801 = 2, então 2.

In [9]:
condicoes = [
    df['J007'] == 2,
    (df['J007'] == 1) & (df['J00801'] == 1),
    (df['J007'] == 1) & (df['J00801'] == 2)
]

# Valores correspondentes às condições
valores = [0, 1, 2]

# Criando novo atributo
df['Doenca_limitante'] = np.select(condicoes, valores, default=np.nan)

In [10]:
print(df['Doenca_limitante'].isna().sum())

0


In [11]:
df = df = df.drop(columns=['J007', 'J00801'])

#### Produto cartesiano N010, N011, N012 e N016

In [90]:
import itertools

# Supondo que essas sejam as categorias possíveis (ajuste conforme o dicionário da PNS)
categorias = [1, 2, 3, 4]  # Por exemplo: 1=Nunca, 2=Raramente, ..., 5=Sempre

# Produto cartesiano
combinacoes = list(itertools.product(categorias, repeat=4))

# Transformar em DataFrame para visualização
import pandas as pd
df_combinacoes = pd.DataFrame(combinacoes, columns=['N010', 'N011', 'N012', 'N016'])

print(df_combinacoes.head())


   N010  N011  N012  N016
0     1     1     1     1
1     1     1     1     2
2     1     1     1     3
3     1     1     1     4
4     1     1     2     1


In [91]:
# Como string: para criar um código tipo "1-2-3"
df['sono_combinado'] = df['N010'].astype(str) + '-' + df['N011'].astype(str) + '-' + df['N012'].astype(str)

# Ou como tupla (útil para agrupar e contar depois)
df['sono_combinado_tuple'] = list(zip(df['N010'], df['N011'], df['N012']))


In [92]:
# Como DataFrame
frequencias = df['sono_combinado'].value_counts().reset_index()
frequencias.columns = ['sono_combinado', 'frequencia']

frequencias


,sono_combinado,frequencia
0,1.0-1.0-1.0,10577
1,2.0-1.0-1.0,1610
2,1.0-2.0-1.0,1309
3,2.0-2.0-2.0,1074
4,2.0-2.0-1.0,922
...,...,...
59,3.0-4.0-3.0,19
60,2.0-1.0-4.0,17
61,2.0-3.0-4.0,16
62,3.0-1.0-4.0,10


In [93]:
df = df.drop(columns=['sono_combinado', 'sono_combinado_tuple'])

#### IMC

In [12]:
# Calculando o IMC (peso / altura²) e convertendo altura de cm para metros
df['imc'] = df['P00104'] / ((df['P00404'] / 100) ** 2)

# Classificando em faixas
conditions = [
    (df['imc'] < 18.5),
    ((df['imc'] >= 18.5) & (df['imc'] < 25)),
    ((df['imc'] >= 25) & (df['imc'] < 30)),
    ((df['imc'] >= 30) & (df['imc'] < 35)),
    ((df['imc'] >= 35) & (df['imc'] < 40)),
    (df['imc'] >= 40)
]

labels = [
    'Abaixo do peso',
    'Peso normal',
    'Sobrepeso',
    'Obesidade Grau I',
    'Obesidade Grau II',
    'Obesidade Grau III'
]

# Criando a coluna com as faixas
df['faixa_imc'] = np.select(conditions, labels, default='Dados inválidos')

# Opcional: Arredondar o IMC para 2 casas decimais
df['imc'] = df['imc'].round(2)

In [13]:
df = df.drop(columns=['P00104', 'P00404'])

#### Produto cartesiano P04501 e P04502

In [96]:
import itertools
import pandas as pd

# Categorias possíveis: opções de 1 a 6
categorias = list(range(1, 7))

# Produto cartesiano
combinacoes = list(itertools.product(categorias, categorias))

# Transformar em DataFrame
df_combinacoes = pd.DataFrame(combinacoes, columns=['P04501', 'P04502'])

print(df_combinacoes)

    P04501  P04502
0        1       1
1        1       2
2        1       3
3        1       4
4        1       5
5        1       6
6        2       1
7        2       2
8        2       3
9        2       4
10       2       5
11       2       6
12       3       1
13       3       2
14       3       3
15       3       4
16       3       5
17       3       6
18       4       1
19       4       2
20       4       3
21       4       4
22       4       5
23       4       6
24       5       1
25       5       2
26       5       3
27       5       4
28       5       5
29       5       6
30       6       1
31       6       2
32       6       3
33       6       4
34       6       5
35       6       6


In [97]:
df['tela_combinado'] = df['P04501'].astype(str) + '-' + df['P04502'].astype(str)

In [98]:
# Criar coluna combinada como string, tipo "1-2", "3-4"
df['tela_combinado'] = df['P04501'].astype(str) + '-' + df['P04502'].astype(str)

# Contar as combinações
frequencias_tela = df['tela_combinado'].value_counts().reset_index()
frequencias_tela.columns = ['tela_combinado', 'frequencia']

# Visualizar o resultado
frequencias_tela


,tela_combinado,frequencia
0,2.0-6.0,3481
1,3.0-6.0,2554
2,1.0-6.0,2136
3,4.0-6.0,1845
4,2.0-1.0,1311
5,6.0-6.0,1285
6,1.0-1.0,1199
7,2.0-2.0,1186
8,3.0-1.0,915
9,3.0-2.0,806


In [99]:
df = df.drop(columns=['tela_combinado'])

#### Discretização da variável contínua Q064 (idade do diagnóstico) em faixas categóricas:

0 - não tem DC
1 - de 0 a 25 anos
2 - de 25 a 40 anos
3 - de 40 a 55 anos
4 - de 55 a 70 anos
5 - > de 70 anos 

In [14]:
def categorizar_dc(idade):
    if idade == 0:
        return 0  # Não tem doença do coração
    elif idade <= 25:
        return 1
    elif idade <= 40:
        return 2
    elif idade <= 55:
        return 3
    elif idade <= 70:
        return 4
    else:
        return 5

df['Q064'] = df['Q064'].apply(categorizar_dc)

In [15]:
df['Q064'].value_counts().sort_index()

Q064
0    22680
1       94
2      261
3      584
4      513
5      186
Name: count, dtype: int64

#### Discretização da variável contínua Renda em faixas categóricas: 

#### Discretização da variável contínua Horas_trabalhadas em faixas categóricas: 

In [16]:
df = df.to_csv('../Bases/Criados/5_plano_cartesiano.csv', index=False)

# Drops não uteis

In [ ]:
# Não tem necessidade
##df = df.drop('V0031', axis=1)

In [165]:
# J01101 Quando ____ consultou um médico pela última vez 
# H001 Quando foi a última vez que o(a) sr(a) consultou com um(a) médico(a)?
# É a mesma coisa e não é util

df = df.drop('J01101', axis=1)
df = df.drop('H001', axis=1)

# Primeira consulta com esse médico?
# Local do atendimento
# Dificuldade para atendimento
# Todas essas tão bem inuteis

df = df.drop('H002', axis=1)
df = df.drop('H004', axis=1)
df = df.drop('H010', axis=1)

In [166]:
# E006011 e E008 dropados por não trazerem muita info util
df = df.drop('E008', axis=1)
df = df.drop('E006011', axis=1)

In [167]:
# Não são muito uteis, eu sei que medem a cobertura do plano mas ta intankavel colocar isso aq
# Titular do plano de saúde
# Nº de ordem do titular
# Plano de servidor público?

# df = df.drop('I00103', axis=1)
# df = df.drop('I001031', axis=1)
# df = df.drop('I004', axis=1)

# Planos Cartesianos

In [168]:
# Juntei todos os atributos que diziam valor de renda que recebiam no renda

# Ajustando os valores conforme as condições especificadas
df['F001021'] = df.apply(lambda x: x['F001021'] if x['F001011'] == 1 else 0, axis=1)
df['F007021'] = df.apply(lambda x: x['F007021'] if x['F007011'] == 1 else 0, axis=1)
df['F008021'] = df.apply(lambda x: x['F008021'] if x['F008011'] == 1 else 0, axis=1)
df['VDF00102'] = df.apply(lambda x: x['VDF00102'] if x['VDF001'] == 1 else 0, axis=1)

# Somando tudo com a renda
df['Renda'] = df['Renda'] + df['F001021'] + df['F007021'] + df['F008021'] + df['VDF00102']

In [169]:
# Dropei os atributos usados anteriormente
df = df.drop('F001021', axis=1)
df = df.drop('F007021', axis=1)
df = df.drop('F008021', axis=1)
df = df.drop('VDF00102', axis=1)
df = df.drop('F001011', axis=1)
df = df.drop('F007011', axis=1)
df = df.drop('F008011', axis=1)
df = df.drop('VDF001', axis=1)

In [170]:
# Criando o novo atributo combinado do plano de saude
# df['Plano_de_Saúde_Cobertura'] = df['I00102'].astype(str)  # Começa com o valor de I00102

# Combinando com I00401, I00402, I00403 apenas quando I00102 for 1
#df.loc[df['I00102'] == 1, 'Plano_de_Saúde_Cobertura'] = (
#   df['I00102'].astype(str) + ' ' + 
#    df['I00401'].astype(str) + ' ' + 
#    df['I00402'].astype(str) + ' ' +  
#    df['I00403'].astype(str)
#)

# Quando I00102 for 2, mantém apenas "2" (não combina com os outros)
#df.loc[df['I00102'] == 2, 'Plano_de_Saúde_Cobertura'] = '2'

In [171]:
# Dropei os atributos usados anteriormente
#df = df.drop('I00401', axis=1)
#df = df.drop('I00402', axis=1)
#df = df.drop('I00403', axis=1)

In [172]:
# Criando a nova coluna com a soma das horas tv e dispositivos
df['horas_dispositivos_tv'] = df['P04501'] + df['P04502']

# Verificando se há valores nulos (opcional)
print("Registros nulos na soma:", df['horas_dispositivos_tv'].isnull().sum())

Registros nulos na soma: 0


In [173]:
df['horas_dispositivos_tv']

0         8.0
1         7.0
2         4.0
3         6.0
4         7.0
         ... 
24340     4.0
24341     8.0
24342     4.0
24343    10.0
24344     8.0
Name: horas_dispositivos_tv, Length: 24345, dtype: float64

In [174]:
# Dropei os atributos usados anteriormente
df = df.drop('P04501', axis=1)
df = df.drop('P04502', axis=1)

# IMC

In [175]:
# calculo do imc, mas eu provavelmente vou mudar ele de lugar, pois precisamos inputar primeiro

import numpy as np

# Calculando o IMC (peso / altura²) e convertendo altura de cm para metros
df['imc'] = df['P00104'] / ((df['P00404'] / 100) ** 2)

# Classificando em faixas
conditions = [
    (df['imc'] < 18.5),
    ((df['imc'] >= 18.5) & (df['imc'] < 25)),
    ((df['imc'] >= 25) & (df['imc'] < 30)),
    ((df['imc'] >= 30) & (df['imc'] < 35)),
    ((df['imc'] >= 35) & (df['imc'] < 40)),
    (df['imc'] >= 40)
]

labels = [
    'Abaixo do peso',
    'Peso normal',
    'Sobrepeso',
    'Obesidade Grau I',
    'Obesidade Grau II',
    'Obesidade Grau III'
]

# Criando a coluna com as faixas
df['faixa_imc'] = np.select(conditions, labels, default='Dados inválidos')

# Opcional: Arredondar o IMC para 2 casas decimais
df['imc'] = df['imc'].round(2)

In [176]:
df['imc']

0        25.88
1        34.05
2        36.89
3        28.38
4        28.96
         ...  
24340    24.84
24341    23.62
24342    22.32
24343    25.51
24344    31.53
Name: imc, Length: 24345, dtype: float64

# Remoção de Registros

In [120]:
#Peso final tinha alguns registros em branco, então eu retirei eles

# Total de registros antes de remover
total_antes = len(df)

# Remover linhas onde P00104 está vazio (NaN ou string vazia)
df = df[~df['P00104'].isna() & (df['P00104'] != '')]

# Total de registros depois de remover
total_depois = len(df)

# Exibir quantos registros foram retirados
registros_removidos = total_antes - total_depois
print(f"Registros removidos: {registros_removidos}")

# Total de registros antes de remover
total_antes = len(df)

TypeError: object of type 'NoneType' has no len()

# Extras

In [178]:
# Verificar colunas com dados faltantes
colunas_com_faltantes = df.columns[df.isnull().any()].tolist()

# Mostrar apenas as colunas com dados faltantes e a quantidade de faltantes
df_faltantes = df[colunas_com_faltantes].isnull().sum()
print(df_faltantes[df_faltantes > 0])

Series([], dtype: float64)


In [69]:
# 4. Salvar em um novo CSV (opcional)
df.to_csv('../Bases/Criados/5_base_plano_cartesiano.csv', index=False)